# Use Case 1: Determining the Peak Trip Date
* Objective: Identify the date with the highest number of completed trips during the two-week period.
* Importance: Understanding peak travel dates helps allocate resources efficiently, optimize driver availability, and prepare for increased demand on specific days.
* Outcome: A report that highlights the busiest day, helping in planning and forecasting for similar future peak dates.

# Use Case 2: Maximum Trips Within a 24-Hour Period
* Objective: Find out the highest number of completed trips in any 24-hour period within the two weeks.
* Importance: Pinpointing the most active 24-hour period allows for better scheduling, ensuring that sufficient drivers are available during the busiest times.
* Outcome: Insights to adjust operational strategies and increase driver availability during peak 24-hour windows.

# Use Case 3: Peak Hour of the Day Analysis
* Objective: Identify which hour of the day experienced the most trip requests throughout the two-week period.
* Importance: This helps optimize the allocation of drivers by concentrating resources during peak demand hours to minimize waiting times for customers.
* Outcome: Data-driven recommendations for driver shifts to ensure coverage during the highest demand periods.
# Use Case 4: Percentage of Zeroes Occurring on Weekends
* Objective: Calculate the percentage of zeroes (unfulfilled requests) that occur during the weekend period.
* Importance: Understanding when and why zeroes occur allows for better planning and improves service availability during critical times, such as weekends.
* Outcome: An actionable report that highlights when zeroes are most likely, aiding in strategy formulation to minimize service disruptions.

# Use Case 5: Weighted Average Ratio of Completed Trips per Driver
Objective: Compute the weighted average ratio of completed trips per driver across the two-week period.
* Importance: Helps in evaluating driver performance and efficiency by accounting for trip volumes at different times, leading to fair scheduling and performance assessments.
* Outcome: A detailed report providing insights into driver productivity and potential areas for optimizing driver allocation.

# Use Case 6: Busiest 8 Consecutive Hours for Scheduling
* Objective: Determine the busiest 8-hour consecutive period for scheduling purposes, considering unique requests.
* Importance: Optimizing driver schedules to cover these critical 8-hour windows ensures maximum coverage and reduces wait times for customers.
* Outcome: Recommendations for an 8-hour shift that maximizes efficiency and driver utilization while meeting peak demand.

# Use Case 7: Correlation Between Driver Supply and Demand
* Objective: Test if driver supply increases in tandem with demand increases during the two-week period.
* Importance: Understanding this correlation allows for more accurate forecasting and ensures that driver supply matches customer demand effectively.
* Outcome: A visual and statistical analysis validating or refuting the hypothesis, providing insights into operational adjustments.

# Use Case 8: Highest Ratio of Zeroes to Eyeballs in a 72-Hour Period
* Objective: Identify the 72-hour period with the highest ratio of zeroes (unmet requests) to eyeballs (requests made).
* Importance: Pinpointing problematic time frames helps improve service quality by analyzing root causes and adjusting strategies for better driver availability during these periods.
* Outcome: Targeted improvement plans for the identified 72-hour windows to reduce zeroes and enhance customer satisfaction.

# Use Case 9: Optimal Hour for Adding 5 Additional Drivers
* Objective: Determine the most critical hour each day to add 5 additional drivers to maximize service efficiency.
* Importance: This proactive measure ensures that the supply of drivers aligns with peak demand, reducing the likelihood of unmet requests and improving overall service quality.
* Outcome: Data-driven decision to optimize driver distribution for peak efficiency.

# Use Case 10: Determining the True "End Day" Time
* Objective: Analyze the data to find a more natural end-of-day time rather than midnight, based on when supply and demand are at their lowest.
* Importance: Establishing an accurate end-of-day time aids in better reporting, scheduling, and operational planning by aligning activities with natural demand patterns.
* Outcome: An optimized schedule based on a data-backed "end day" time, improving resource management and operational efficiency.

In [44]:
# importing necessary liberaries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd


In [45]:
spark=SparkSession.builder.appName("Uber Data Analysis").getOrCreate()

In [46]:
uber_data=spark.read.csv("uber_data.csv",header=True,inferSchema=True)
uber_data.show()

+---------+------------+---------+-------+----------------+---------+--------------+
|     Date|Time (Local)|Eyeballs |Zeroes |Completed Trips |Requests |Unique Drivers|
+---------+------------+---------+-------+----------------+---------+--------------+
|10-Sep-12|           7|        5|      0|               2|        2|             9|
|     NULL|           8|        6|      0|               2|        2|            14|
|     NULL|           9|        8|      3|               0|        0|            14|
|     NULL|          10|        9|      2|               0|        1|            14|
|     NULL|          11|       11|      1|               4|        4|            11|
|     NULL|          12|       12|      0|               2|        2|            11|
|     NULL|          13|        9|      1|               0|        0|             9|
|     NULL|          14|       12|      1|               0|        0|             9|
|     NULL|          15|       11|      2|               1|      

In [47]:
from pyspark.sql.functions import col, to_date, last
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

# # 1. Handle Missing Dates (forward-fill)
window_spec = Window.orderBy("Time (Local)").rowsBetween(Window.unboundedPreceding, 0)
uber_data = uber_data.withColumn("Date", last("Date", True).over(window_spec))

# 2. Convert the Date column to the correct format
uber_data = uber_data.withColumn("Date", to_date(col("Date"), "dd-MMM-yy"))

# 3. Ensure proper data types for other columns
uber_data = uber_data.withColumn("Time_Local", col("Time (Local)").cast(IntegerType())) \
       .withColumn("Eyeballs", col("Eyeballs ").cast(IntegerType())) \
       .withColumn("Zeroes", col("Zeroes ").cast(IntegerType())) \
       .withColumn("Completed_Trips", col("Completed Trips ").cast(IntegerType())) \
       .withColumn("Requests", col("Requests ").cast(IntegerType())) \
       .withColumn("Unique_Drivers", col("Unique Drivers").cast(IntegerType())) \
       .drop("Time (Local)","Eyeballs ","Zeroes ","Completed Trips ","Requests ","Unique Drivers")

# Show the cleaned DataFrame
uber_data.orderBy("Date").show()


+----------+----------+--------+------+---------------+--------+--------------+
|      Date|Time_Local|Eyeballs|Zeroes|Completed_Trips|Requests|Unique_Drivers|
+----------+----------+--------+------+---------------+--------+--------------+
|2012-09-10|         7|       5|     0|              2|       2|             9|
|2012-09-10|         8|       8|     2|              1|       2|             3|
|2012-09-10|         7|      10|     0|              2|       2|             5|
|2012-09-10|         7|       5|     2|              2|       2|             4|
|2012-09-10|         7|       7|     3|              0|       0|             4|
|2012-09-10|         7|      10|     1|              1|       1|             4|
|2012-09-10|         7|       9|     1|              2|       2|             5|
|2012-09-10|         7|       9|     7|              1|       1|             1|
|2012-09-10|         7|       3|     0|              0|       0|             1|
|2012-09-10|         7|      11|     4| 

In [48]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [49]:
# # rename columns
renamed_data=uber_data.withColumnRenamed("Date","Date_of_Trip") \
         .withColumnRenamed("Time (Local)","Time_Local") \
         .withColumnRenamed("Completed Trips","Completed_Trips") \
         .withColumnRenamed("Unique Drivers","Unique_Drivers")

renamed_data.show()


+------------+----------+--------+------+---------------+--------+--------------+
|Date_of_Trip|Time_Local|Eyeballs|Zeroes|Completed_Trips|Requests|Unique_Drivers|
+------------+----------+--------+------+---------------+--------+--------------+
|  2012-09-11|         0|       9|     3|              1|       1|             3|
|  2012-09-12|         0|       9|     2|              0|       1|             2|
|  2012-09-13|         0|      11|    11|              0|       2|             0|
|  2012-09-14|         0|      10|     1|              3|       4|             3|
|  2012-09-15|         0|      45|     2|             23|      24|            19|
|  2012-09-16|         0|      44|     2|             17|      20|            15|
|  2012-09-17|         0|      11|     5|              0|       2|             2|
|  2012-09-18|         0|      28|    18|              3|      13|             4|
|  2012-09-19|         0|       9|     1|              0|       0|             3|
|  2012-09-20|  

In [50]:
# usecase: 1
# Group the data by date and sum the completed trips
completed_trips_by_date = renamed_data.filter(col('Date_of_Trip').isNotNull()) \
                                      .groupBy("Date_of_Trip").sum("Completed_Trips") \
                                      .withColumnRenamed("sum(Completed_Trips)", "Total_Completed_Trips") \
                                      .orderBy("Total_Completed_Trips",ascending=False) \
                                      .first()

print(f'''The date with maximum numbers of trip is {completed_trips_by_date['Date_of_Trip']} \
      The toatl numbers of trips completed were {completed_trips_by_date['Total_Completed_Trips']}''')


The date with maximum numbers of trip is 2012-09-10       The toatl numbers of trips completed were 1085


In [51]:
# usecase 2:
from pyspark.sql.functions import sum, window

# Group the data by 24-hour windows and sum the completed trips
completed_trips_by_window = renamed_data \
    .groupBy("Time_Local") \
    .agg(sum("Completed_Trips").alias("Total_Completed_Trips")) \
    .orderBy("Total_Completed_Trips", ascending=False)

# # Get the highest number of completed trips within a 24-hour period
highest_completed_trips_in_24_hours = completed_trips_by_window \
    .select("Total_Completed_Trips") \
    .first()["Total_Completed_Trips"]
completed_trips_by_window.show(1)



+----------+---------------------+
|Time_Local|Total_Completed_Trips|
+----------+---------------------+
|        22|                  134|
+----------+---------------------+
only showing top 1 row



In [52]:
#usecase 3:
hourly_requests = renamed_data.groupBy("Time_Local") \
                              .sum("Requests") \
                              .withColumnRenamed("sum(Requests)", "total_requests") \
                              .orderBy("total_requests", ascending=False)


most_requested_hour = hourly_requests.select("Time_Local").first()[0]
print("The hour with the most requests is:", most_requested_hour)

The hour with the most requests is: 23


In [53]:
# Usecase 4:
from pyspark.sql.functions import dayofweek, col, sum

# # Filter for weekend (Saturday and Sunday) and the time range (after 5 PM or before 3 AM)
weekend_zeros = renamed_data.filter(
    ((col("Time_Local") >= 17) | (col("Time_Local") < 3)) &  # Time filter
    ((dayofweek("Date_of_Trip") == 7) | (dayofweek("Date_of_Trip") == 1))  # Weekend filter: Saturday (7) and Sunday (1)
).agg(sum("Zeroes").alias("weekend_zeros")).collect()[0]["weekend_zeros"]

# Calculate total zeros
total_zeros = renamed_data.agg(sum("Zeroes").alias("total_zeros")).collect()[0]["total_zeros"]


# Calculate the percentage of weekend zeros
percent_weekend_zeros = (weekend_zeros / total_zeros) * 100

print("The percentage of zeros that occurred on weekends is:", percent_weekend_zeros, "%")


The percentage of zeros that occurred on weekends is: 2.309307207837649 %


In [54]:
weighted_avg = renamed_data.withColumn("completed_per_driver", renamed_data["Completed_Trips"] / renamed_data["Unique_Drivers"]) \
                 .groupBy("Date_of_Trip", "Time_Local") \
                 .agg(avg("completed_per_driver").alias("avg_completed_per_driver"), sum("Completed_Trips").alias("total_completed_trips")) \
                 .withColumn("weighted_ratio", col("avg_completed_per_driver") * col("total_completed_trips")) \
                 .agg(sum("weighted_ratio") / sum("total_completed_trips")).collect()[0][0]

print("The weighted average ratio of completed trips per driver is:", weighted_avg)


The weighted average ratio of completed trips per driver is: 0.6009636007707119


In [55]:
# Usecase no 6:
# Calculate the number of unique requests for each hour of the day
hourly_unique_requests = (renamed_data
  .groupBy("Time_Local")
  .agg(countDistinct("Requests").alias("unique_requests"))
)

# Slide a window of 8 hours to find the busiest 8 consecutive hours
window = Window.orderBy(col("unique_requests").desc()).rowsBetween(0, 7)
busiest_8_consecutive_hours = (hourly_unique_requests
  .select("*", sum("unique_requests").over(window).alias("consecutive_sum"))
  .orderBy(col("consecutive_sum").desc())
  .limit(1)
)

# Print the result
busiest_8_consecutive_hours.show()

+----------+---------------+---------------+
|Time_Local|unique_requests|consecutive_sum|
+----------+---------------+---------------+
|        20|             12|             80|
+----------+---------------+---------------+



In [ ]:
#Usecase 8:

# Group the data by 72-hour periods and calculate the ratio of zeroes to eyeballs for each period
period_ratios = (renamed_data
  .groupBy(((col("Date_of_Trip").cast("timestamp").cast("long") / (72*3600)).cast("int")).alias("period"))
  .agg(sum("Zeroes").alias("zeroes"), sum("Eyeballs").alias("eyeballs"))
  .withColumn("ratio", col("zeroes") / col("eyeballs"))
)

# Find the period with the highest ratio
highest_ratio_period = period_ratios.orderBy(col("ratio").desc()).limit(1)

# Print the result
highest_ratio_period.show()

In [ ]:
# Usecase 9:
# Calculate requests per unique driver for each hour
requests_per_driver = (renamed_data.groupBy('Time_Local').agg(
    (sum('Requests') / countDistinct('Unique_Drivers')).alias('requests_per_driver'))
)

# Show the hour with the highest ratio
requests_per_driver.orderBy(desc('requests_per_driver')).show(1)

In [ ]:
# Usecase 10:
# Calculate average completed trips and unique drivers for each hour
avg_trips_and_drivers = (renamed_data.groupBy('Time_Local').agg(
    mean('Completed_Trips').alias('avg_completed_trips'),
    mean('Unique_Drivers').alias('avg_unique_drivers')
))

# Show the hour with the lowest average completed trips and unique drivers
avg_trips_and_drivers.orderBy('avg_completed_trips', 'avg_unique_drivers').show(1)